In [ ]:
###### 📱 Material Design Scientific Calculator - UNIFIED INPUT
# ========================================================

from ipywidgets import widgets, Layout, GridspecLayout
from IPython.display import display
import math
import threading

class ScientificCalculatorEnhanced:
    def __init__(self):
        self.current_expression = ""
        self.display_text = "0"
        self.result_calculated = False
    
    def append_to_expression(self, value: str) -> str:
        if self.result_calculated:
            if value in ['+', '-', '*', '/', '^']:
                self.current_expression = self.display_text
            else:
                self.current_expression = ""
            self.result_calculated = False
        
        self.current_expression += str(value)
        return self.current_expression
    
    def calculate(self) -> str:
        try:
            expression = self.current_expression
            if not expression:
                return self.display_text
                
            allowed_chars = "0123456789.+-*/()^√πsin cos tan log ln !"
            if not all(c in allowed_chars for c in expression.replace(" ", "")):
                raise ValueError("Invalid characters")
            
            import math
            eval_dict = {
                'sin': math.sin, 'cos': math.cos, 'tan': math.tan,
                'log': math.log10, 'ln': math.log, 'sqrt': math.sqrt,
                'π': math.pi, 'e': math.e, 'factorial': math.factorial,
                'math': math, 'abs': abs, 'pow': pow
            }
            
            expression = expression.replace('^', '**').replace('√', 'sqrt')
            import re
            expression = re.sub(r'(\d+)!', r'factorial(\1)', expression)
            
            result = eval(expression, {"__builtins__": {}}, eval_dict)
            
            if isinstance(result, float):
                if result.is_integer():
                    self.display_text = str(int(result))
                else:
                    self.display_text = str(round(result, 10)).rstrip('0').rstrip('.')
            else:
                self.display_text = str(result)
            
            self.current_expression = self.display_text
            self.result_calculated = True
            return self.display_text
            
        except Exception as e:
            self.display_text = "Error"
            self.current_expression = ""
            self.result_calculated = False
            return "Error"
    
    def clear(self):
        self.current_expression = ""
        self.display_text = "0"
        self.result_calculated = False
        return self.display_text
    
    def delete_last(self):
        if self.current_expression:
            self.current_expression = self.current_expression[:-1]
        return ""
    
    def get_display(self):
        return self.display_text

# Material Design Colors
COLORS = {
    'primary': '#1976D2', 'primary_dark': '#1565C0', 'accent': '#FF9800',
    'text_primary': '#212121', 'surface': '#FFFFFF',
    'error': '#F44336', 'light_blue': '#E3F2FD'
}

class JupyterMaterialCalculator:
    def __init__(self):
        self.calc = ScientificCalculatorEnhanced()
        
        # Store button references for keyboard triggering and animation
        self.buttons = {}
        
        # **NO TEXTBOX** - Removed as requested
        
        # Expression display - shows full expression being built
        self.expression_display = widgets.HTML(
            value=f"<div style='text-align:right; font-size:18px; color:#E3F2FD; " \
                  f"background-color:{COLORS['primary_dark']}; padding:12px 24px; " \
                  f"font-family:Roboto; min-height:24px; border-radius:12px 12px 0 0;'>" \
                  f"</div>"
        )
        
        # Main result display
        self.display = widgets.HTML(
            value=f"<div style='text-align:right; font-size:44px; padding:30px; " \
                  f"background: linear-gradient(135deg, {COLORS['primary']}, {COLORS['primary_dark']}); " \
                  f"color:white; border-radius:0 0 12px 12px; font-family:Roboto; " \
                  f"font-weight:500; min-height:120px;'>{self.calc.get_display()}</div>"
        )
        
        self.create_ui()
        self.setup_keyboard_events()
        
    def create_ui(self):
        def create_btn(label, btn_type='number'):
            style = {
                'number': {'bg': COLORS['surface'], 'color': COLORS['text_primary']},
                'operator': {'bg': COLORS['primary'], 'color': 'white'},
                'scientific': {'bg': COLORS['light_blue'], 'color': COLORS['primary_dark']},
                'clear': {'bg': COLORS['error'], 'color': 'white'},
                'equals': {'bg': COLORS['accent'], 'color': 'white'},
                'delete': {'bg': COLORS['primary_dark'], 'color': 'white'}
            }[btn_type]
            
            btn = widgets.Button(
                description=label,
                layout=Layout(width='auto', height='75px', margin='3px'),
                style={'button_color': style['bg'], 'text_color': style['color'],
                       'font_weight': 'bold', 'font_size': '20px'}
            )
            return btn
        
        # Create all buttons and store references
        btn_defs = [
            ('sin', 'scientific'), ('cos', 'scientific'), ('tan', 'scientific'),
            ('C', 'clear'), ('←', 'delete'),
            ('log', 'scientific'), ('ln', 'scientific'), ('√', 'scientific'),
            ('^', 'scientific'), ('!', 'scientific'),
            ('7', 'number'), ('8', 'number'), ('9', 'number'),
            ('/', 'operator'), ('π', 'scientific'),
            ('4', 'number'), ('5', 'number'), ('6', 'number'),
            ('*', 'operator'), ('(', 'scientific'),
            ('1', 'number'), ('2', 'number'), ('3', 'number'),
            ('-', 'operator'), (')', 'scientific'),
            ('0', 'number'), ('.', 'number'), ('=', 'equals'), ('+', 'operator')
        ]
        
        for label, btn_type in btn_defs:
            self.buttons[label] = create_btn(label, btn_type)
        
        # Configure button handlers
        for key, btn in self.buttons.items():
            if key == '=':
                btn.on_click(lambda b: self.on_equals())
            elif key == 'C':
                btn.on_click(lambda b: self.on_clear())
            elif key == '←':
                btn.on_click(lambda b: self.on_delete())
            else:
                btn.on_click(lambda b, k=key: self.on_button_clicked(k))
        
        # Grid layout
        grid = GridspecLayout(7, 5, width='100%', height='auto')
        
        # Place displays
        grid[0, :] = self.expression_display
        grid[1, :] = self.display
        
        # Button placements
        placements = [
            (2, ['sin', 'cos', 'tan', 'C', '←']),
            (3, ['log', 'ln', '√', '^', '!']),
            (4, ['7', '8', '9', '/', 'π']),
            (5, ['4', '5', '6', '*', '(']),
            (6, ['1', '2', '3', '-', ')']),
        ]
        
        for row_idx, btn_row in placements:
            for col_idx, btn_key in enumerate(btn_row):
                grid[row_idx, col_idx] = self.buttons[btn_key]
        
        # Bottom row (manual layout for spans)
        bottom_row = widgets.HBox([
            self.buttons['0'],
            self.buttons['.'],
            self.buttons['+']
        ], layout=Layout(width='100%'))
        self.buttons['0'].layout = Layout(width='35%', height='75px', margin='3px')
        self.buttons['.'].layout = Layout(width='15%', height='75px', margin='3px')
        self.buttons['+'].layout = Layout(width='15%', height='75px', margin='3px')
        
        # Equals row
        equals_row = widgets.HBox([self.buttons['=']], layout=Layout(width='100%'))
        self.buttons['='].layout = Layout(width='100%', height='75px', margin='3px')
        
        # Main container
        self.main_container = widgets.VBox([
            grid,
            bottom_row,
            equals_row
        ], layout=Layout(width='520px', margin='20px auto', 
                        box_shadow='0 10px 25px rgba(0,0,0,0.15)',
                        border_radius='12px',
                        overflow='hidden',
                        background='white'))
        
        display(self.main_container)
        
    def setup_keyboard_events(self):
        """Keyboard input triggers button clicks with animation"""
        
        def flash_button(button, duration=0.15):
            """Animate button press: change to accent color then back"""
            original_bg = button.style.button_color
            original_text = button.style.text_color
            
            # Change to accent color (orange)
            button.style.button_color = COLORS['accent']
            button.style.text_color = 'white'
            
            # Restore after delay
            def restore():
                button.style.button_color = original_bg
                button.style.text_color = original_text
            
            threading.Timer(duration, restore).start()
        
        def on_value_change(change):
            # This is a hidden text input that captures keyboard events
            # It's not displayed in the UI, but captures key presses
            pass  # We'll use a different approach
        
        # **REAL SOLUTION**: Use a hidden Text widget to capture keyboard
        # This widget is created but not displayed - it just captures key events
        self.hidden_input = widgets.Text(
            value='',
            placeholder='',
            layout=Layout(width='0px', height='0px', visibility='hidden')
        )
        
        def on_hidden_change(change):
            raw_value = change['new']
            if not raw_value:
                return
            
            # **ENTER KEY HANDLING**
            if '\n' in raw_value or '\r' in raw_value:
                self.on_equals()
                self.hidden_input.value = ''
                # Flash equals button
                flash_button(self.buttons['='])
                return
            
            # Process the character
            char = raw_value[-1]
            
            # Map keyboard characters to button keys
            char_to_button = {
                '0': '0', '1': '1', '2': '2', '3': '3', '4': '4',
                '5': '5', '6': '6', '7': '7', '8': '8', '9': '9',
                '.': '.', '+': '+', '-': '-', '*': '*', '/': '/',
                '^': '^', '(': '(', ')': ')', '!': '!', 'π': 'π', 'p': 'π',
                's': 'sin', 'c': 'cos', 't': 'tan', 'l': 'log'
            }
            
            if char in char_to_button:
                button_key = char_to_button[char]
                if button_key in self.buttons:
                    # Trigger the button click
                    self.on_button_clicked(button_key)
                    # Flash the button for visual feedback
                    flash_button(self.buttons[button_key])
            elif char == '\x7f':  # Backspace
                self.on_delete()
                # Flash delete button
                flash_button(self.buttons['←'])
            
            # Clear hidden input
            self.hidden_input.value = ''
        
        self.hidden_input.observe(on_hidden_change, names='value')
        
        # Focus the hidden input to capture keystrokes
        threading.Timer(0.2, lambda: self.hidden_input.focus()).start()
        
        # **NO VISIBLE TEXTBOX** - Removed as requested
    
    def on_button_clicked(self, key):
        """Called when button is clicked by mouse OR triggered by keyboard"""
        if key in ['+', '-', '*', '/', '^']:
            self.calc.append_to_expression(key)
        else:
            self.calc.append_to_expression(key)
        self.update_display()
    
    def on_equals(self):
        self.calc.calculate()
        self.update_display()
    
    def on_clear(self):
        self.calc.clear()
        self.update_display()
    
    def on_delete(self):
        self.calc.delete_last()
        self.update_display()
    
    def update_display(self):
        # Update expression display (top blue area)
        expr = self.calc.current_expression
        self.expression_display.value = f"<div style='text-align:right; font-size:18px; color:#E3F2FD; " \
                                       f"background-color:{COLORS['primary_dark']}; padding:12px 24px; " \
                                       f"font-family:Roboto; min-height:24px; border-radius:12px 12px 0 0;'>" \
                                       f"{expr}</div>"
        
        # Update result display (bottom blue area)
        display_val = self.calc.get_display()
        self.display.value = f"<div style='text-align:right; font-size:44px; padding:30px; " \
                           f"background: linear-gradient(135deg, {COLORS['primary']}, {COLORS['primary_dark']}); " \
                           f"color:white; border-radius:0 0 12px 12px; font-family:Roboto; " \
                           f"font-weight:500; min-height:120px;'>{display_val}</div>"

# Launch
print("🧮 Material Design Scientific Calculator - FINAL UNIFIED VERSION")
print("=" * 70)
print("✅ NO TEXTBOX - Removed as requested")
print("✅ Expression area shows full expression")
print("✅ Keyboard input triggers button animations")
print("✅ Click buttons OR type - both update expression")
print("✅ Press ENTER to calculate")
print("=" * 70)

In [39]:
calc = JupyterMaterialCalculator()